In [1]:
import os
import openai
import time
import traceback
import json
from tqdm.auto import tqdm
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

c:\Users\gooog\anaconda3\envs\nlp_clean\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_base = os.environ.get("OPENAI_API_BASE")
if "s" not in openai.api_key:
    print("KEY LOADING FAILED!")
    raise NameError

In [3]:
def prompt_api(model, eval_prompt1, eval_prompt2):
    for i in range(10):
        try:
            eval_step1 = openai.ChatCompletion.create(
                model=model,
                stream=False,
                temperature=0,
                messages=[{"role": "user", "content": eval_prompt1}],
            )
            break
        except Exception as exc:
            print(traceback.format_exc())
            print(exc)
            if i != 9:
                print(f"Retrying... (i = {i})")
            eval_step1 = {"choices": [{"message": {"content": "ERROR IN STEP 1"}}]}
            time.sleep(3)
    if "ERROR" in eval_step1["choices"][0]["message"]["content"]:
        eval_step2 = {"choices": [{"message": {"content": "ERROR IN STEP 1"}}]}
        return eval_step1, eval_step2
    for i in range(10):
        try:
            eval_step2 = openai.ChatCompletion.create(
                model=model,
                stream=False,
                temperature=0,
                messages=[
                    {"role": "user", "content": eval_prompt1},
                    {"role": "assistant", "content": eval_step1.choices[0].message.content},
                    {"role": "user", "content": eval_prompt2},
                ]
            )
            break
        except Exception as exc:
            print(traceback.format_exc())
            print(exc)
            if i != 9:
                print(f"Retrying... (i = {i})")
            eval_step2 = {"choices": [{"message": {"content": "ERROR IN STEP 2"}}]}
            time.sleep(3)
    return eval_step1, eval_step2

In [4]:
def get_eval_prompts(answer, lang):
    if lang == "en":
        eval_prompt1 = f"""In the following exercise, the student is given a beginning of a story. The student needs to complete it into a full story.
The exercise tests the student's language abilities and creativity. The symbol *** marks the separator between the prescribed beginning and the student's completion:
{answer["prompt"]}***{answer["completion"]}
Please provide your general assessment about the part written by the student (the one after the *** symbol).
Is it gramatically correct? Is it consistent with the beginning of the story? Pay special attention to whether the student manages to complete the sentence which is split in the middle by the separator ***."""
        eval_prompt2 = f"""Now, grade the student’s completion in terms of grammar, creativity, consistency with the story’s beginning and whether the plot makes sense.
Moreover, please provide your best guess of what the age of the student might be, as reflected from the completion.
Choose from possible age groups: A: 3 or under. B: 4-5. C: 6-7. D: 8-9. E: 10-12. F: 13-16.
Use the format below replacing # and X with your answer and do not write anything more than that:
grammar: #/10, creativity: #/10, consistency: #/10, age: X"""
    else:
        eval_prompt1 = f"""In der folgenden Übung bekommt der Schüler den Anfang einer Geschichte vorgegeben. Der Schüler muss sie zu einer vollständigen Geschichte ausarbeiten.
Die Übung testet die Sprachfähigkeiten und die Kreativität des Schülers. Das Symbol *** markiert die Trennung zwischen dem vorgegebenen Anfang und dem vom Schüler vervollständigten Teil:
{answer["prompt"]}***{answer["completion"]}
Bitte geben Sie Ihre allgemeine Beurteilung über den vom Schüler geschriebenen Teil (den Teil nach dem *** Symbol) ab.
Ist er grammatikalisch korrekt? Ist er konsistent mit dem Anfang der Geschichte? Achten Sie besonders darauf, ob der Schüler den Satz vervollständigt, der in der Mitte durch das Trennzeichen *** geteilt wird."""
        eval_prompt2 = f"""Jetzt bewerten Sie bitte den vom Schüler vervollständigten Teil in Bezug auf Grammatik, Kreativität, Konsistenz mit dem Anfang der Geschichte und ob die Handlung Sinn ergibt.
Darüber hinaus geben Sie bitte Ihre beste Einschätzung ab, wie alt der Schüler sein könnte, basierend auf der Vervollständigung.
Wählen Sie aus möglichen Altersgruppen: A: 3 oder jünger. B: 4-5. C: 6-7. D: 8-9. E: 10-12. F: 13-16.
Benutzen Sie das folgende Format, wobei Sie # und X mit Ihrer Antwort ersetzen, und sonst nichts weiteres schreiben.
Grammatik: #/10, Kreativität: #/10, Konsistenz: #/10, Alter: X"""
    return eval_prompt1, eval_prompt2

In [5]:
def get_single_eval(eval_model, answer, lang):
    eval_prompt1, eval_prompt2 = get_eval_prompts(answer, lang)
    eval_step1, eval_step2 = prompt_api(eval_model, eval_prompt1, eval_prompt2)
    answer["eval_step1"] = eval_step1["choices"][0]["message"]["content"]
    answer["eval_step2"] = eval_step2["choices"][0]["message"]["content"]
    return answer

In [6]:
def get_evals(eval_model, lang, model_id):
    with open(f"answers_{lang}_{model_id}.json", "r") as f:
        answers = json.load(f)

    evaluations = []
    for a in tqdm(answers):
        try:
            ev = get_single_eval(eval_model, a, lang)
        except Exception as e:
            print("error in get_single_eval")
            ev = a
            ev["eval_step1"] = "ERROR: SOMETHING WENT WRONG"
            ev["eval_step2"] = "ERROR: SOMETHING WENT WRONG"
        evaluations.append(ev)
    with open(f"evaluations_{lang}_{model_id}.json", "w") as f:
        json.dump(evaluations, f, indent="    ", ensure_ascii=False)

In [8]:
eval_model = "gpt-4-0125-preview"
lang = "en"
# for model_id in ["gpt", "ts", "own_small"]:
for model_id in ["own_small"]:
    print(f"Evaluating english {model_id}-model.")
    get_evals(eval_model, lang, model_id)

lang = "de"
# for model_id in ["gpt", "own_small", "own_big"]:
for model_id in ["own_small"]:
    print(f"Evaluating german {model_id}-model.")
    get_evals(eval_model, lang, model_id)

Evaluating english own_small-model.


100%|██████████| 100/100 [38:18<00:00, 22.99s/it]


Evaluating german own_small-model.


100%|██████████| 100/100 [58:34<00:00, 35.15s/it]
